In [9]:
import pandas as pd
import numpy as np
import ta  # technical analysis library
from datetime import datetime

In [10]:
raw_SPY_intraday_cleaned_df = pd.read_csv("..\Data\SPY_intraday_cleaned.csv")

In [11]:
raw_SPY_intraday_cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184552 entries, 0 to 184551
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   timestamp_utc  184552 non-null  object 
 1   open           184552 non-null  float64
 2   high           184552 non-null  float64
 3   low            184552 non-null  float64
 4   close          184552 non-null  float64
 5   volume         184552 non-null  float64
 6   vwap           184552 non-null  float64
 7   ret            184552 non-null  float64
 8   vol_5m         184552 non-null  float64
 9   vol_30m        184552 non-null  float64
dtypes: float64(9), object(1)
memory usage: 14.1+ MB


# Feature Preprocessing

## Add features

In [14]:
# Copy dataset
SPY_intraday_cleaned_df = raw_SPY_intraday_cleaned_df.copy()

# Time format
SPY_intraday_cleaned_df['timestamp_utc'] = pd.to_datetime(SPY_intraday_cleaned_df['timestamp_utc'])
SPY_intraday_cleaned_df = SPY_intraday_cleaned_df.set_index('timestamp_utc').sort_index()

price_cols = ['open', 'high', 'low', 'close', 'volume']

# Momentum
SPY_intraday_cleaned_df['log_ret'] = np.log(SPY_intraday_cleaned_df['close'] / SPY_intraday_cleaned_df['close'].shift(1))

# Rolling returns
SPY_intraday_cleaned_df['ret_15m'] = SPY_intraday_cleaned_df['close'].pct_change(3)   # assuming 5min data → 3 intervals = 15min
SPY_intraday_cleaned_df['ret_30m'] = SPY_intraday_cleaned_df['close'].pct_change(6)

# Volatility
SPY_intraday_cleaned_df['vol_15m'] = SPY_intraday_cleaned_df['log_ret'].rolling(3).std()
SPY_intraday_cleaned_df['vol_60m'] = SPY_intraday_cleaned_df['log_ret'].rolling(12).std()

# Trend
SPY_intraday_cleaned_df['MA_short'] = SPY_intraday_cleaned_df['close'].rolling(12).mean()   # 1 hour
SPY_intraday_cleaned_df['MA_long'] = SPY_intraday_cleaned_df['close'].rolling(60).mean()    # 5 hours
SPY_intraday_cleaned_df['MA_ratio'] = SPY_intraday_cleaned_df['MA_short'] / SPY_intraday_cleaned_df['MA_long']   # relative trend

# Technical Indicators
SPY_intraday_cleaned_df['MACD'] = ta.trend.macd(SPY_intraday_cleaned_df['close'])
SPY_intraday_cleaned_df['RSI'] = ta.momentum.rsi(SPY_intraday_cleaned_df['close'], window=14)
SPY_intraday_cleaned_df['ATR'] = ta.volatility.average_true_range(SPY_intraday_cleaned_df['high'], SPY_intraday_cleaned_df['low'], SPY_intraday_cleaned_df['close'], window=14)


# Time features
SPY_intraday_cleaned_df['hour'] = SPY_intraday_cleaned_df.index.hour
SPY_intraday_cleaned_df['dayofweek'] = SPY_intraday_cleaned_df.index.dayofweek

SPY_intraday_cleaned_df['hour_sin'] = np.sin(2 * np.pi * SPY_intraday_cleaned_df['hour'] / 24)
SPY_intraday_cleaned_df['hour_cos'] = np.cos(2 * np.pi * SPY_intraday_cleaned_df['hour'] / 24)
SPY_intraday_cleaned_df['dow_sin'] = np.sin(2 * np.pi * SPY_intraday_cleaned_df['dayofweek'] / 7)
SPY_intraday_cleaned_df['dow_cos'] = np.cos(2 * np.pi * SPY_intraday_cleaned_df['dayofweek'] / 7)

SPY_intraday_cleaned_df = SPY_intraday_cleaned_df.dropna().reset_index()

SPY_intraday_cleaned_df.head()


,timestamp_utc,open,high,low,close,volume,vwap,ret,vol_5m,vol_30m,...,MA_ratio,MACD,RSI,ATR,hour,dayofweek,hour_sin,hour_cos,dow_sin,dow_cos
0,2024-01-02 15:29:00+00:00,471.6500,471.9499,471.6500,471.7600,108515.0,471.8182,0.000212,0.000404,0.000307,...,0.999885,-0.088302,40.882336,0.248887,15,1,-0.707107,-0.707107,0.781831,0.62349
1,2024-01-02 15:30:00+00:00,471.7700,471.7700,471.4899,471.5000,223224.0,471.6311,-0.000551,0.000435,0.000309,...,0.999814,-0.119714,35.413989,0.251116,15,1,-0.707107,-0.707107,0.781831,0.62349
2,2024-01-02 15:31:00+00:00,471.5050,471.5493,471.4500,471.4799,196061.0,471.5029,-0.000043,0.000346,0.000306,...,0.999754,-0.144565,35.023964,0.240272,15,1,-0.707107,-0.707107,0.781831,0.62349
3,2024-01-02 15:32:00+00:00,471.4796,471.5200,471.4000,471.4113,377918.0,471.4616,-0.000145,0.000297,0.000306,...,0.999672,-0.167859,33.661381,0.231681,15,1,-0.707107,-0.707107,0.781831,0.62349
4,2024-01-02 15:33:00+00:00,471.4100,471.5499,471.4097,471.4696,114603.0,471.4955,0.000124,0.000298,0.000306,...,0.999578,-0.179546,35.942240,0.225147,15,1,-0.707107,-0.707107,0.781831,0.62349


In [16]:
# ===========================================================
# 🔹 TARGET CREATION — Directional Labels for Next 5-Minute Move
# ===========================================================

# Define future return over next 5-minute window
SPY_intraday_cleaned_df['future_ret_5m'] = SPY_intraday_cleaned_df['close'].shift(-1) / SPY_intraday_cleaned_df['close'] - 1

# Binary target: 1 if next close > current close (price goes up), else 0
SPY_intraday_cleaned_df['y_next5_up'] = (SPY_intraday_cleaned_df['future_ret_5m'] > 0).astype(int)


# Drop rows where future return cannot be computed (the last row(s))
SPY_intraday_cleaned_df = SPY_intraday_cleaned_df.dropna(subset=['future_ret_5m']).reset_index(drop=True)

print("✅ Target column added:")
print(SPY_intraday_cleaned_df[['timestamp_utc', 'close', 'future_ret_5m', 'y_next5_up']].tail(10))


✅ Target column added:
                   timestamp_utc     close  future_ret_5m  y_next5_up
184481 2025-10-22 20:49:00+00:00  667.7200       0.000150           1
184482 2025-10-22 20:50:00+00:00  667.8200      -0.000045           0
184483 2025-10-22 20:51:00+00:00  667.7900      -0.000105           0
184484 2025-10-22 20:52:00+00:00  667.7200       0.000000           0
184485 2025-10-22 20:53:00+00:00  667.7200      -0.000105           0
184486 2025-10-22 20:54:00+00:00  667.6500       0.000000           0
184487 2025-10-22 20:55:00+00:00  667.6500       0.000060           1
184488 2025-10-22 20:56:00+00:00  667.6900       0.000015           1
184489 2025-10-22 20:57:00+00:00  667.6999       0.000075           1
184490 2025-10-22 20:58:00+00:00  667.7500       0.000135           1


# Train test split